<a href="https://colab.research.google.com/github/vicente-gonzalez-ruiz/denoising/blob/main/src/denoising/SRA_volume_denoising_opticalflow3d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SRA_empiar10311

In [ ]:
#%pip install ipynbname

In [ ]:
#%pip install gdown

In [ ]:
#%pip install matplotlib

In [ ]:
# %pip install --force-reinstall "information_theory @ git+https://github.com/vicente-gonzalez-ruiz/information_theory"
#%pip install "information_theory @ git+https://github.com/vicente-gonzalez-ruiz/information_theory"

In [ ]:
#%pip install cupy-cuda12x # Check the CUDA installed with nvcc --version (that should match with the version provided by nvidia-smi)
#%pip install numba

In [ ]:
#%pip install --force-reinstall "motion_estimation @ git+https://github.com/vicente-gonzalez-ruiz/motion_estimation"
#%pip uninstall -y motion_estimation
#%pip install "motion_estimation @ git+https://github.com/vicente-gonzalez-ruiz/motion_estimation"

In [ ]:
#%pip install --force-reinstall "optical_flow_3D @ git+https://github.com/vicente-gonzalez-ruiz/optical_flow_3D"
#%pip uninstall -y optical_flow_3D
#%pip install "optical_flow_3D @ git+https://github.com/vicente-gonzalez-ruiz/optical_flow_3D"

In [ ]:
!ln -sf ~/repos/denoising/src/denoising .

In [ ]:
import mrcfile
import numpy as np
import os
import mrcfile
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from collections import namedtuple
import logging
import motion_estimation._3D.farneback_optical_flow_3D as farneback3D
import denoising.volume.SRA_optical_flow_3D as SRA
import motion_estimation._3D.project_optical_flow_3D as project
from PIL import Image
import ipynbname

In [ ]:
vol_filename = "empiar11415.mrc"
Args = namedtuple("args", ["input", "output"])
fn, fe = vol_filename.split(".")
args = Args(vol_filename , "denoised.mrc")
print(args)

In [ ]:
logging.basicConfig(format="[%(filename)s:%(lineno)s %(funcName)s()] %(message)s")
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
logger.setLevel(logging.WARNING)

In [ ]:
OF_estimator = farneback3D.OF_Estimation(
    logger=logger,
    block_size=(512, 512, 512),
    overlap=(64, 64, 64)
)

In [ ]:
projector = project.Project(logger)

In [ ]:
!ln -sf /nas/vruiz/vols/empiar11415.mrc .

In [ ]:
#%pip install mrcfile
stack_MRC = mrcfile.open(args.input)
noisy = stack_MRC.data#[:, 0:512, 0:512]
noisy = (255*(noisy - np.min(noisy))/(np.max(noisy) - np.min(noisy))).astype(np.uint8)
print(noisy.shape)

In [ ]:
def get_notebook_name():
    ip = get_ipython()
    this_notebook_path = ip.user_ns['__session__']
    this_notebook_name = os.path.basename(this_notebook_path)
    return this_notebook_name, this_notebook_path

In [ ]:
get_notebook_name()

In [ ]:
def show_image(denoised, title):
    slice_idx = denoised.shape[0]//2
    figure(figsize=(32, 32))
    plt.subplot(1, 2, 1)
    plt.title("denoised")
    plt.imshow(denoised[slice_idx][::-1, :], cmap="gray")
    plt.subplot(1, 2, 2)
    plt.title("difference")
    plt.imshow(noisy[slice_idx][::-1, :] - denoised[slice_idx][::-1, :], cmap="gray")
    plt.show()

In [ ]:
class Output_Controller:

    def __init__(self):
        self.image_counter = 0
    
    def show_image(self, denoised, title):
        slice_idx = denoised.shape[0]//2
        image_float = denoised[slice_idx]
        image_8b = (255*(image_float - np.min(image_float))/(np.max(image_float) - np.min(image_float))).astype(np.uint8)
        image = Image.fromarray(image_8b)
        notebook_name = get_notebook_name()[0]
        output_filename = f"{notebook_name}_{self.image_counter}.png"
        image.save(output_filename)
        self.image_counter += 1
        figure(figsize=(32, 32))
        plt.subplot(1, 2, 1)
        plt.title("denoised")
        plt.imshow(denoised[slice_idx][::-1, :], cmap="gray")
        plt.subplot(1, 2, 2)
        plt.title("difference")
        plt.imshow(noisy[slice_idx][::-1, :] - denoised[slice_idx][::-1, :], cmap="gray")
        plt.show()

OC = Output_Controller()

In [ ]:
denoiser = SRA.Shuffle_Register_and_Average(
    OF_estimator,
    projector,
    logger,
    quality_estimator=None,
    show_image=OC.show_image)

In [ ]:
denoised = denoiser.filter(
    noisy_volume=noisy,
    N_iters=100,
    std_dev=2.0,
    pyramid_levels=1,
    gauss_size=5,
    iterations=5,
    sigma_k=3,
    #filter_type="box", # box | gaussian
    filter_type="gaussian", # box | gaussian
    filter_size=9, # std_dev=2.0
    presmoothing=None
)

In [ ]:
with mrcfile.new("denoised.mrc", overwrite=True) as mrc:
    mrc.set_data(denoised.astype(np.float32))
    mrc.data

In [ ]:
denoised = mrcfile.open("denoised.mrc").data

In [ ]:
denoised.shape

In [ ]:
figure(figsize=(16, 16))
slice_idx = denoised.shape[0]//2
plt.imshow(denoised[slice_idx, 200:600, 200:600], cmap="gray")

In [ ]:
figure(figsize=(16, 16))
slice_idx = noisy.shape[0]//2
plt.imshow(noisy[slice_idx, 200:600, 200:600], cmap="gray")

In [ ]:
import information_theory
slice_index = noisy.shape[0]//2
fig, axs = plt.subplots(1, 3, figsize=(15, 45))
axs[0].imshow(noisy[16], cmap="gray")
axs[0].set_title(f"Noisy")
axs[1].imshow(denoised[slice_index], cmap="gray")
axs[1].set_title(f"Denoised (DQI={information_theory.information.compute_quality_index(noisy[slice_index], denoised[slice_index])})")
axs[2].imshow(noisy[slice_index] - denoised[slice_index], cmap="gray")
axs[2].set_title(f"Noisy - Denoised")
fig.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16, 32))
slice_idx = noisy.shape[1]//2
axs[0].imshow(noisy[:, slice_idx], cmap="gray")
axs[0].set_title(f"Noisy")
axs[1].imshow(denoised[:, slice_idx], cmap="gray")
axs[1].set_title(f"Denoised (DQI={information_theory.information.compute_quality_index(noisy[:, slice_idx], denoised[:, slice_idx])})")
fig.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16, 32))
slice_idx = noisy.shape[2]//2
axs[0].imshow(noisy[:, :, slice_idx], cmap="gray")
axs[0].set_title(f"Noisy")
axs[1].imshow(denoised[:, :, slice_idx], cmap="gray")
axs[1].set_title(f"Denoised (DQI={information_theory.information.compute_quality_index(noisy[:, :, slice_idx], denoised[:, :, slice_idx])})")
fig.tight_layout()
plt.show()

In [ ]:
slice_idx = noisy.shape[0] // 2

fig, axes = plt.subplots(1, 2, figsize=(20, 20))

# Plot the original slice z
im1 = axes[0].imshow(noisy[slice_idx, 0:512, 0:512].T, cmap='gray', origin='lower')
axes[0].set_title(f'Original Slice Z={slice_idx}')
axes[0].grid(False)

# Plot the original slice z+1
im2 = axes[1].imshow(denoised[slice_idx, 0:512, 0:512].T, cmap='gray', origin='lower')
axes[1].set_title(f'N2N-Odd-Even-Registered Denoised Slice Z={slice_idx}')
axes[1].grid(False)

plt.tight_layout()
plt.show()